In [1]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar
import pickle
from selenium import webdriver

ModuleNotFoundError: No module named 'selenium'

In [2]:
company_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000789019&type=&dateb=&owner=include&count=40'

In [ ]:
def crawl_by_company(company_url,browser,financial_report_type):
        
#     options = webdriver.ChromeOptions()
#     prefs = {'download.default_directory': './data'}
#     options.add_experimental_option('prefs', prefs)
#     browser=webdriver.Chrome(chrome_filepath,options=options) 
    browser.get(company_url)
    browser.find_element_by_name('type').send_keys(financial_report_type)
    browser.find_element_by_xpath("//input[@value='Search']").click()
    time.sleep(1)
    b_url = browser.current_url
    
    finance_report_url = []
    res = requests.get(b_url)
    doc = pq(res.text)

    temp = 'https://www.sec.gov'
    for i,d in enumerate(doc('.tableFile2 #documentsbutton').items()):

        f_url = temp+d.attr('href')    
        browser.get(f_url)
        lv1_res = requests.get(f_url)
        lv1_doc = pq(lv1_res.text)
        txt_num = lv1_doc('#secNum').text().split(' ')[-1]
        browser.find_element_by_xpath(f"//a[contains(text(),'{txt_num}.txt')] ").click()
        time.sleep(1.5)
        print(browser.current_url)
        finance_report_url.append(browser.current_url)
    
#     browser.quit()
    
    return finance_report_url

In [ ]:
def crawl_10_Q_url(company_url,browser,financial_report_type):
    browser.get(company_url)
    browser.find_element_by_name('type').send_keys(financial_report_type)
    browser.find_element_by_xpath("//input[@value='Search']").click()
    time.sleep(1)
    return browser.current_url

In [ ]:
from lxml import etree

In [ ]:
def crawl_fp_txt_url(company_url):
    finance_report_url = []
    res = requests.get(company_url)
    doc = pq(res.text)

    temp = 'https://www.sec.gov'
    for i,d in enumerate(doc('.tableFile2 #documentsbutton').items()):

        f_url = temp+d.attr('href')    
        lv1_res = requests.get(f_url)
        lv1_doc = pq(lv1_res.text)
        txt_num = lv1_doc('#secNum').text().split(' ')[-1]
        sel = etree.HTML(lv1_res.text)
        txt_url = sel.xpath(f"//a[contains(text(),'{txt_num}.txt')] ")[0].attrib['href']
        finance_report_url.append(txt_url)
        
    if (len(finance_report_url) == 0):
        finance_report_url = 'Unfounded'
        
    return finance_report_url

In [47]:
company_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000789019&type=&dateb=&owner=include&count=40'

In [48]:
finance_report_url = crawl_by_company(company_url,'/Users/lou/Downloads/chromedriver','10-K')

https://www.sec.gov/Archives/edgar/data/789019/000156459018019062/0001564590-18-019062.txt
https://www.sec.gov/Archives/edgar/data/789019/000156459017014900/0001564590-17-014900.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312516662209/0001193125-16-662209.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312515272806/0001193125-15-272806.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312514289961/0001193125-14-289961-index.htm
https://www.sec.gov/Archives/edgar/data/789019/000119312513310206/0001193125-13-310206.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312512316848/0001193125-12-316848.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312511200680/0001193125-11-200680.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312510171791/0001193125-10-171791.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312509158735/0001193125-09-158735.txt
https://www.sec.gov/Archives/edgar/data/789019/000119312508162768/0001193125-08-1627

In [4]:
def get_finace_report_txt(finance_report_url):
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    
    for url in finance_report_url:
        symbol = url.split('/')[6] # number of company
        year = url.split('-')[1]  # 94 ~ 18
        filename = './data/' + symbol + '_' + year + 'txt'
        urllib.request.urlretrieve(url,filename)
        print(filename)

In [85]:
get_finace_report_txt(finance_report_url)

./data/789019_18txt
./data/789019_17txt
./data/789019_16txt
./data/789019_15txt
./data/789019_14txt
./data/789019_13txt
./data/789019_12txt
./data/789019_11txt
./data/789019_10txt
./data/789019_09txt
./data/789019_08txt
./data/789019_07txt
./data/789019_06txt
./data/789019_05txt
./data/789019_04txt
./data/789019_03txt
./data/789019_02txt
./data/789019_01txt
./data/789019_00txt
./data/789019_99txt
./data/789019_98txt
./data/789019_97txt
./data/789019_96txt
./data/789019_95txt
./data/789019_94txt


In [5]:
tickers = pd.read_csv('all_ticker.csv').iloc[:,1].values
tickers

array(['FSLA', 'DSKE', 'CHIR', ..., 'CERO', 'ATRS', 'VIPTS'], dtype=object)

In [6]:
tickers[0]

'FSLA'

In [7]:
def get_search_addr(chrome_path,tickers):
    
    files = os.listdir()
    max_ver = 0

    for f in files:
        if 'search' in f:
            version = int(f.split('_')[2].split('.')[0])
            if (version > max_ver ):
                max_ver = version
                max_f = f
    if (max_ver == 0):     
        search_addr = {}
    else:
        with open(max_f,'rb') as f:
            search_addr = pickle.load(f)
    length = len(search_addr)
    
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(chrome_path,options=options) 
    
    for i,tic in enumerate(tickers[length:]):
        browser.get('https://www.sec.gov/edgar/searchedgar/legacy/companysearch.html')
        browser.find_element_by_xpath("//input[@title='CIK']").send_keys(tic)
        browser.find_element_by_xpath("//input[@value='Find Companies']  ").click()
        c_url = browser.current_url
#         time.sleep(1)
        
        res = requests.get(c_url)
        doc = pq(res.text)
        message = doc('center').text()
        if (message == 'No matching Ticker Symbol.'):
#             print('Unfound')
            search_addr[tic] = 'Unfound'
        else:
            search_addr[tic] = c_url
            
        if(len(search_addr)%1000 == 0):
            num = len(search_addr)/1000
            
            with open(f"search_addr_{num}.pkl","wb") as f:
                pickle.dump(search_addr,f)
                print(f'Save search_addr_{num}.pkl',i+length)
    browser.quit()
    
    with open("search_addr_final.pkl","wb") as f:
        pickle.dump(search_addr,f)
        print('Save search_addr_final.pkl',i+length)
                
    return search_addr

In [71]:
seach_addr = get_search_addr('/Users/lou/Downloads/chromedriver',tickers)

Save search_addr_7.0.pkl 6999
Save search_addr_8.0.pkl 7999
Save search_addr_9.0.pkl 8999
Save search_addr_10.0.pkl 9999
Save search_addr_11.0.pkl 10999
Save search_addr_12.0.pkl 11999
Save search_addr_13.0.pkl 12999
Save search_addr_14.0.pkl 13999
Save search_addr_15.0.pkl 14999
Save search_addr_16.0.pkl 15999
Save search_addr_17.0.pkl 16999
Save search_addr_18.0.pkl 17999
Save search_addr_19.0.pkl 18999
Save search_addr_20.0.pkl 19999
Save search_addr_21.0.pkl 20999
Save search_addr_22.0.pkl 21999
Save search_addr_final.pkl 22975


In [8]:
with open('search_addr_final.pkl','rb') as f:
    search_addr = pickle.load(f)

In [35]:
def get_all_fp_url(tickers,company_addr):
    # check directory
    if not os.path.isdir('./fp_url'):
        os.mkdir('./fp_url')
    
    # check files
    files = os.listdir('./fp_url')
    max_ver = 0

    for f in files:
        if 'company' in f:
            version = int(f.split('_')[3].split('.')[0])
            if (version > max_ver ):
                max_ver = version
                max_f = f
    if (max_ver == 0):     
        company_fp_dic = {}
    else:
        if (os.path.getsize(f"./fp_url/{max_f}") > 0):
            with open(f'./fp_url/{max_f}','rb') as f:
                company_fp_dic = pickle.load(f)
        else:
            company_fp_dic = {}
    length = len(company_fp_dic)   
    
    for i,key in enumerate(tickers[length:]):
        addr = search_addr[key]
        
        if (addr != 'Unfound'):
            fp_addr = crawl_fp_txt_url(addr)
            company_fp_dic[key] = fp_addr
        else:
            company_fp_dic[key] = 'Unfounded'
            
        if(len(company_fp_dic)%1000 == 0):
            num = len(company_fp_dic)/1000

            with open(f"./fp_url/company_fp_dic_{num}.pkl","wb") as f:
                pickle.dump(company_fp_dic,f)
                print(f'./fp_url/company_fp_dic_{num}.pkl',i+length)
                
    with open(f"./fp_url/company_fp_dic_final.pkl","wb") as f:
        pickle.dump(company_fp_dic,f)
        print(f'./fp_url/company_fp_dic_final.pkl',i+length)

In [36]:
def get_all_ft_url(tickers,search_addr,chrome_filepath,financial_report_type):
    # check directory
    if not os.path.isdir(f'./{financial_report_type}_url'):
        os.mkdir(f'./{financial_report_type}_url')
    
    # check files
    files = os.listdir(f'./{financial_report_type}_url')
    max_ver = 0

    for f in files:
        if 'company' in f:
            version = int(f.split('_')[3].split('.')[0])
            if (version > max_ver ):
                max_ver = version
                max_f = f
    if (max_ver == 0):     
        company_ft_dic = {}
    else:
        # Check file size > 0
        if (os.path.getsize(f"./{financial_report_type}_url/{max_f}") > 0):
            with open(f'./{financial_report_type}_url/{max_f}','rb') as f:
                company_ft_dic = pickle.load(f)
        else:
            company_ft_dic = {}
    length = len(company_ft_dic)
    
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': f'./{financial_report_type}_url'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(chrome_filepath,options=options) 
    
    for i,key in enumerate(tickers[length:]):
        addr = search_addr[key]
        
        if (addr != 'Unfound'):
            ft_addr = crawl_10_Q_url(addr,browser,financial_report_type)
            company_ft_dic[key] = ft_addr
        else:
            company_ft_dic[key] = 'Unfounded'
            
        if(len(company_ft_dic)%1000 == 0):
            num = len(company_ft_dic)/1000
            
            with open(f"./{financial_report_type}_url/company_ft_dic_{num}.pkl","wb") as f:
                pickle.dump(company_ft_dic,f)
                print(f'./{financial_report_type}_url/company_ft_dic_{num}.pkl',i+length)
    with open(f"./{financial_report_type}_url/company_ft_dic_final.pkl","wb") as f:
        pickle.dump(company_ft_dic,f)
        print(f'./{financial_report_type}_url/company_ft_dic_final.pkl',i+length)

In [9]:
search_addr

{'FSLA': 'Unfound',
 'DSKE': 'https://www.sec.gov/cgi-bin/browse-edgar?company=&match=&CIK=DSKE&filenum=&State=&Country=&SIC=&owner=exclude&Find=Find+Companies&action=getcompany',
 'CHIR': 'https://www.sec.gov/cgi-bin/browse-edgar?company=&match=&CIK=CHIR&filenum=&State=&Country=&SIC=&owner=exclude&Find=Find+Companies&action=getcompany',
 'NIV': 'https://www.sec.gov/cgi-bin/browse-edgar?company=&match=&CIK=NIV&filenum=&State=&Country=&SIC=&owner=exclude&Find=Find+Companies&action=getcompany',
 'TNO': 'Unfound',
 'GWGH': 'https://www.sec.gov/cgi-bin/browse-edgar?company=&match=&CIK=GWGH&filenum=&State=&Country=&SIC=&owner=exclude&Find=Find+Companies&action=getcompany',
 'IV': 'Unfound',
 'MERI': 'Unfound',
 'PLYM': 'https://www.sec.gov/cgi-bin/browse-edgar?company=&match=&CIK=PLYM&filenum=&State=&Country=&SIC=&owner=exclude&Find=Find+Companies&action=getcompany',
 'MERR': 'https://www.sec.gov/cgi-bin/browse-edgar?company=&match=&CIK=MERR&filenum=&State=&Country=&SIC=&owner=exclude&Find=

In [37]:
get_all_ft_url(tickers,search_addr,'/Users/lou/Downloads/chromedriver','10-Q')

./10-Q_url/company_ft_dic_final.pkl 22975


In [75]:
get_all_fp_url(tickers,search_addr,'/Users/lou/Downloads/chromedriver','10-Q')

https://www.sec.gov/Archives/edgar/data/1642453/000155837018008662/0001558370-18-008662.txt
https://www.sec.gov/Archives/edgar/data/1642453/000155837018006913/0001558370-18-006913.txt
https://www.sec.gov/Archives/edgar/data/1642453/000155837018004675/0001558370-18-004675.txt
https://www.sec.gov/Archives/edgar/data/1642453/000155837017008737/0001558370-17-008737.txt
https://www.sec.gov/Archives/edgar/data/1642453/000155837017006437/0001558370-17-006437.txt
https://www.sec.gov/Archives/edgar/data/1642453/000155837017004101/0001558370-17-004101.txt
https://www.sec.gov/Archives/edgar/data/1642453/000121390016017854/0001213900-16-017854.txt
https://www.sec.gov/Archives/edgar/data/1642453/000121390016015099/0001213900-16-015099.txt
https://www.sec.gov/Archives/edgar/data/1642453/000121390016013312/0001213900-16-013312.txt
https://www.sec.gov/Archives/edgar/data/1642453/000121390015008432/0001213900-15-008432.txt
https://www.sec.gov/Archives/edgar/data/1642453/000121390015006739/0001213900-15

https://www.sec.gov/Archives/edgar/data/1515816/000117152015000311/0001171520-15-000311.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152014000740/0001171520-14-000740.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152014000587/0001171520-14-000587.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152014000372/0001171520-14-000372.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152013000714/0001171520-13-000714.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152013000525/0001171520-13-000525.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152013000405/0001171520-13-000405.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152012000980/0001171520-12-000980.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152012000979/0001171520-12-000979.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152012000954/0001171520-12-000954.txt
https://www.sec.gov/Archives/edgar/data/1515816/000117152012000793/0001171520-12

https://www.sec.gov/Archives/edgar/data/96885/000110889007000032/0001108890-07-000032.txt
https://www.sec.gov/Archives/edgar/data/96885/000089109206000393/0000891092-06-000393.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012311074944/0000950123-11-074944.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012311047927/0000950123-11-047927.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012310103337/0000950123-10-103337.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012310074832/0000950123-10-074832.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012310046899/0000950123-10-046899.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012309059881/0000950123-09-059881.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095012309032604/0000950123-09-032604.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095013709003808/0000950137-09-003808.txt
https://www.sec.gov/Archives/edgar/data/1299688/000095013708013590/0000950137-08-013

https://www.sec.gov/Archives/edgar/data/943605/000095013704007598/0000950137-04-007598.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013704004747/0000950137-04-004747.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013704001419/0000950137-04-001419.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013704001418/0000950137-04-001418.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013404002837/0000950134-04-002837.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013404002836/0000950134-04-002836.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013404002835/0000950134-04-002835.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013403016667/0000950134-03-016667.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013403012765/0000950134-03-012765.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013403009233/0000950134-03-009233.txt
https://www.sec.gov/Archives/edgar/data/943605/000095013402015767/0000950134-02-015767.txt

https://www.sec.gov/Archives/edgar/data/1166388/000095012310059309/0000950123-10-059309.txt
https://www.sec.gov/Archives/edgar/data/1166388/000095012310056921/0000950123-10-056921.txt
https://www.sec.gov/Archives/edgar/data/1166388/000090951805000976/0000909518-05-000976.txt
https://www.sec.gov/Archives/edgar/data/1166388/000090951805000969/0000909518-05-000969.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752305008048/0001157523-05-008048.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752305005394/0001157523-05-005394.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752304011304/0001157523-04-011304.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752304008383/0001157523-04-008383.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752304005545/0001157523-04-005545.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752303007257/0001157523-03-007257.txt
https://www.sec.gov/Archives/edgar/data/1166388/000115752303004779/0001157523-03

...

https://www.sec.gov/Archives/edgar/data/1520504/000110465918026761/0001104659-18-026761.txt
https://www.sec.gov/Archives/edgar/data/1520504/000115752318000763/0001157523-18-000763.txt
https://www.sec.gov/Archives/edgar/data/1520504/000152050418000017/0001520504-18-000017.txt
https://www.sec.gov/Archives/edgar/data/1520504/000152050418000015/0001520504-18-000015.txt
https://www.sec.gov/Archives/edgar/data/1520504/000152050418000013/0001520504-18-000013.txt
https://www.sec.gov/Archives/edgar/data/1520504/000115752318000387/0001157523-18-000387.txt
https://www.sec.gov/Archives/edgar/data/1513137/000114420418008818/0001144204-18-008818.txt
https://www.sec.gov/Archives/edgar/data/1520504/000119312518040090/0001193125-18-040090.txt
https://www.sec.gov/Archives/edgar/data/1520504/000152050418000011/0001520504-18-000011.txt
https://www.sec.gov/Archives/edgar/data/1520504/000119312518034767/0001193125-18-034767.txt
https://www.sec.gov/Archives/edgar/data/1520504/000152050418000008/0001520504-18

https://www.sec.gov/Archives/edgar/data/768411/000089710117001300/0000897101-17-001300.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117001199/0000897101-17-001199.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000927/0000897101-17-000927.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000880/0000897101-17-000880.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000638/0000897101-17-000638.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000632/0000897101-17-000632.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000544/0000897101-17-000544.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000468/0000897101-17-000468.txt
https://www.sec.gov/Archives/edgar/data/768411/000089710117000408/0000897101-17-000408.txt
https://www.sec.gov/Archives/edgar/data/863436/000086343618000042/0000863436-18-000042.txt
https://www.sec.gov/Archives/edgar/data/863436/000086343618000035/0000863436-18-000035.txt

https://www.sec.gov/Archives/edgar/data/1070304/000119312518210349/0001193125-18-210349.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518206504/0001193125-18-206504.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518206495/0001193125-18-206495.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518177492/0001193125-18-177492.txt
https://www.sec.gov/Archives/edgar/data/1070304/000108514618001377/0001085146-18-001377.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518156697/0001193125-18-156697.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518121836/0001193125-18-121836.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518049473/0001193125-18-049473.txt
https://www.sec.gov/Archives/edgar/data/315066/000031506618001479/0000315066-18-001479.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518041108/0001193125-18-041108.txt
https://www.sec.gov/Archives/edgar/data/1070304/000119312518041099/0001193125-18-

https://www.sec.gov/Archives/edgar/data/878736/000129993307005985/0001299933-07-005985.txt
https://www.sec.gov/Archives/edgar/data/878736/000131325007000079/0001313250-07-000079.txt
https://www.sec.gov/Archives/edgar/data/878736/000129993307005825/0001299933-07-005825.txt
https://www.sec.gov/Archives/edgar/data/878736/000095015207007912/0000950152-07-007912.txt
https://www.sec.gov/Archives/edgar/data/355648/000095013707015098/0000950137-07-015098.txt
https://www.sec.gov/Archives/edgar/data/878736/000095012307012617/0000950123-07-012617.txt
https://www.sec.gov/Archives/edgar/data/355648/000095013707013922/0000950137-07-013922.txt
https://www.sec.gov/Archives/edgar/data/878736/000129993307005200/0001299933-07-005200.txt
https://www.sec.gov/Archives/edgar/data/878736/000129993307005201/0001299933-07-005201.txt
https://www.sec.gov/Archives/edgar/data/812295/000081229507000119/0000812295-07-000119.txt
https://www.sec.gov/Archives/edgar/data/792094/000092838502003090/0000928385-02-003090.txt

ConnectionError: HTTPSConnectionPool(host='www.sec.gov', port=443): Max retries exceeded with url: /Archives/edgar/data/768408/000110465913008472/0001104659-13-008472-index.htm (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x126a97828>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))